#**ASSIGNMENT 2: VECTOR ADDITION**

**1] Install cuda toolkit of nvidia**

In [ ]:
!apt-get update
!apt-get install -y --no-install-recommends nvidia-cuda-toolkit

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,784 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,735 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

**2] Set environment variables**

In [ ]:
!export PATH=/usr/local/cuda/bin/:$PATH
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64/:$LD_LIBRARY_PATH


**3] Sequential code of vector addition:**

In [ ]:
%%writefile sequential.cpp

#include <iostream>
#include <Eigen/Dense>
#include <vector>
#include <cmath>
#include <ctime>

using namespace Eigen;
using namespace std;

vector<vector<double>> polar_to_cartesian(const vector<double>& magnitudes, const vector<double>& angles) {
    vector<vector<double>> cartesian_vectors;
    for (size_t i = 0; i < magnitudes.size(); ++i) {
        double x = magnitudes[i] * cos(angles[i] * M_PI / 180.0);
        double y = magnitudes[i] * sin(angles[i] * M_PI / 180.0);
        cartesian_vectors.push_back({x, y});
    }
    return cartesian_vectors;
}

pair<vector<double>, vector<double>> cartesian_to_polar(const MatrixXd& vectors) {
    vector<double> magnitudes;
    vector<double> angles;
    for (int i = 0; i < vectors.rows(); ++i) {
        double magnitude = vectors.row(i).norm();
        double angle = atan2(vectors(i, 1), vectors(i, 0)) * 180.0 / M_PI;
        magnitudes.push_back(magnitude);
        angles.push_back(angle);
    }
    return {magnitudes, angles};
}

pair<vector<double>, vector<double>> add_polar_vectors(const vector<double>& magnitudes1, const vector<double>& angles1,
                                                       const vector<double>& magnitudes2, const vector<double>& angles2) {
    MatrixXd vectors1 = Map<const MatrixXd>(polar_to_cartesian(magnitudes1, angles1)[0].data(), magnitudes1.size(), 2);
    MatrixXd vectors2 = Map<const MatrixXd>(polar_to_cartesian(magnitudes2, angles2)[0].data(), magnitudes2.size(), 2);
    MatrixXd result_vectors = vectors1 + vectors2;
    return cartesian_to_polar(result_vectors);
}

int main() {
    srand(time(0)); // Seed for random number generation

    // Example polar vectors for 1000 pairs
    vector<double> magnitudes_a(1000);
    vector<double> angles_a(1000);
    vector<double> magnitudes_b(1000);
    vector<double> angles_b(1000);

    // Generate random vectors
    for (int i = 0; i < 1000; ++i) {
        magnitudes_a[i] = (rand() % 10) + 1;  // Random magnitudes for vector A
        angles_a[i] = (rand() % 360);         // Random angles for vector A
        magnitudes_b[i] = (rand() % 10) + 1;  // Random magnitudes for vector B
        angles_b[i] = (rand() % 360);         // Random angles for vector B
    }

    clock_t start_time = clock();
    auto result = add_polar_vectors(magnitudes_a, angles_a, magnitudes_b, angles_b);
    clock_t end_time = clock();
    double time_taken = double(end_time - start_time) / CLOCKS_PER_SEC;

    // Print the time taken
    cout << "Time taken for adding 1000 pairs of vectors: " << time_taken << " seconds" << endl;

    // Print the results
    for (int i = 0; i < 1000; ++i) {
        cout << "Pair " << i+1 << ":" << endl;
        cout << "Vector A: Magnitude=" << magnitudes_a[i] << ", Angle=" << angles_a[i] << " degrees" << endl;
        cout << "Vector B: Magnitude=" << magnitudes_b[i] << ", Angle=" << angles_b[i] << " degrees" << endl;
        cout << "Resultant Vector: Magnitude=" << result.first[i] << ", Angle=" << result.second[i] << " degrees" << endl << endl;
    }

    return 0;
}

Writing sequential.cpp


**4] Install the development files for the Eigen C++ template library**

In [ ]:
!apt-get install libeigen3-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  libeigen3-doc libmpfrc++-dev
The following NEW packages will be installed:
  libeigen3-dev
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,056 kB of archives.
After this operation, 9,081 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libeigen3-dev all 3.4.0-2ubuntu2 [1,056 kB]
Fetched 1,056 kB in 1s (758 kB/s)
Selecting previously unselected package libeigen3-dev.
(Reading database ... 123701 files and directories currently installed.)
Preparing to unpack .../libeigen3-dev_3.4.0-2ubuntu2_all.deb ...
Unpacking libeigen3-dev (3.4.0-2ubuntu2) ...
Setting up libeigen3-dev (3.4.0-2ubuntu2) ...


**5] Compile and Run**

In [ ]:
!g++ -o sequential sequential.cpp -I/usr/include/eigen3
!./sequential

Time taken for adding 1000 pairs of vectors: 0.00209 seconds
Pair 1:
Vector A: Magnitude=10, Angle=271 degrees
Vector B: Magnitude=10, Angle=63 degrees
Resultant Vector: Magnitude=5.12005, Angle=22.9599 degrees

Pair 2:
Vector A: Magnitude=7, Angle=33 degrees
Vector B: Magnitude=8, Angle=42 degrees
Resultant Vector: Magnitude=1.09343, Angle=-174.507 degrees

Pair 3:
Vector A: Magnitude=6, Angle=73 degrees
Vector B: Magnitude=2, Angle=290 degrees
Resultant Vector: Magnitude=5.73783, Angle=-90 degrees

Pair 4:
Vector A: Magnitude=1, Angle=67 degrees
Vector B: Magnitude=2, Angle=85 degrees
Resultant Vector: Magnitude=1.75423, Angle=90 degrees

Pair 5:
Vector A: Magnitude=2, Angle=93 degrees
Vector B: Magnitude=9, Angle=221 degrees
Resultant Vector: Magnitude=11.3547, Angle=131.458 degrees

Pair 6:
Vector A: Magnitude=7, Angle=155 degrees
Vector B: Magnitude=9, Angle=128 degrees
Resultant Vector: Magnitude=3.54298, Angle=-55.7939 degrees

Pair 7:
Vector A: Magnitude=8, Angle=182 degrees
Ve

**7] Parallel code of vector addition(using cuda)**

In [ ]:
%%writefile parallel_cuda.cu
#include <iostream>
#include <vector>
#include <cmath>
#include <chrono>

// CUDA kernel to add polar vectors
__global__ void add_polar_vectors_kernel(const double* magnitudes_a, const double* angles_a,
                                         const double* magnitudes_b, const double* angles_b,
                                         double* result_magnitudes, double* result_angles, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        // Convert polar vectors to Cartesian vectors
        double x1 = magnitudes_a[idx] * cos(angles_a[idx] * M_PI / 180.0);
        double y1 = magnitudes_a[idx] * sin(angles_a[idx] * M_PI / 180.0);
        double x2 = magnitudes_b[idx] * cos(angles_b[idx] * M_PI / 180.0);
        double y2 = magnitudes_b[idx] * sin(angles_b[idx] * M_PI / 180.0);

        // Perform vector addition
        double x_sum = x1 + x2;
        double y_sum = y1 + y2;

        // Convert the resultant vectors back to polar coordinates
        result_magnitudes[idx] = sqrt(x_sum * x_sum + y_sum * y_sum);
        result_angles[idx] = atan2(y_sum, x_sum) * 180.0 / M_PI;
    }
}

int main() {
    // Example polar vectors for 1000 pairs
    const int n = 1000;
    std::vector<double> magnitudes_a(n);
    std::vector<double> angles_a(n);
    std::vector<double> magnitudes_b(n);
    std::vector<double> angles_b(n);

    // Generate random vectors
    for (int i = 0; i < n; ++i) {
        magnitudes_a[i] = (rand() % 10) + 1;  // Random magnitudes for vector A
        angles_a[i] = (rand() % 360);         // Random angles for vector A
        magnitudes_b[i] = (rand() % 10) + 1;  // Random magnitudes for vector B
        angles_b[i] = (rand() % 360);         // Random angles for vector B
    }

    // Device arrays
    double *d_magnitudes_a, *d_angles_a, *d_magnitudes_b, *d_angles_b, *d_result_magnitudes, *d_result_angles;
    cudaMalloc(&d_magnitudes_a, n * sizeof(double));
    cudaMalloc(&d_angles_a, n * sizeof(double));
    cudaMalloc(&d_magnitudes_b, n * sizeof(double));
    cudaMalloc(&d_angles_b, n * sizeof(double));
    cudaMalloc(&d_result_magnitudes, n * sizeof(double));
    cudaMalloc(&d_result_angles, n * sizeof(double));

    // Copy data from host to device
    cudaMemcpy(d_magnitudes_a, magnitudes_a.data(), n * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_angles_a, angles_a.data(), n * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_magnitudes_b, magnitudes_b.data(), n * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_angles_b, angles_b.data(), n * sizeof(double), cudaMemcpyHostToDevice);

    // Calculate block and grid dimensions
    int blockSize = 256;
    int numBlocks = (n + blockSize - 1) / blockSize;

    // Measure time for vector addition
    auto start_time = std::chrono::steady_clock::now();

    // Call CUDA kernel
    add_polar_vectors_kernel<<<numBlocks, blockSize>>>(d_magnitudes_a, d_angles_a, d_magnitudes_b, d_angles_b, d_result_magnitudes, d_result_angles, n);

    // Measure time taken
    auto end_time = std::chrono::steady_clock::now();
    double time_taken = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time).count() / 1e6;

    // Copy result from device to host
    std::vector<double> result_magnitudes(n);
    std::vector<double> result_angles(n);
    cudaMemcpy(result_magnitudes.data(), d_result_magnitudes, n * sizeof(double), cudaMemcpyDeviceToHost);
    cudaMemcpy(result_angles.data(), d_result_angles, n * sizeof(double), cudaMemcpyDeviceToHost);

    // Print the time taken
    std::cout << "Time taken for adding 1000 pairs of vectors in parallel: " << time_taken << " seconds" << std::endl;

    // Print the results
    for (int i = 0; i < n; ++i) {
        std::cout << "Pair " << i+1 << ":" << std::endl;
        std::cout << "Vector A: Magnitude=" << magnitudes_a[i] << ", Angle=" << angles_a[i] << " degrees" << std::endl;
        std::cout << "Vector B: Magnitude=" << magnitudes_b[i] << ", Angle=" << angles_b[i] << " degrees" << std::endl;
        std::cout << "Resultant Vector: Magnitude=" << result_magnitudes[i] << ", Angle=" << result_angles[i] << " degrees" << std::endl << std::endl;
    }

    // Free device memory
    cudaFree(d_magnitudes_a);
    cudaFree(d_angles_a);
    cudaFree(d_magnitudes_b);
    cudaFree(d_angles_b);
    cudaFree(d_result_magnitudes);
    cudaFree(d_result_angles);

    return 0;
}

Overwriting parallel_cuda.cu


**8] Compile and Run**

In [ ]:
!nvcc -o parallel_cuda parallel_cuda.cu
!./parallel_cuda

Time taken for adding 1000 pairs of vectors in parallel: 0.16612 seconds
Pair 1:
Vector A: Magnitude=4, Angle=286 degrees
Vector B: Magnitude=8, Angle=115 degrees
Resultant Vector: Magnitude=4.09731, Angle=123.785 degrees

Pair 2:
Vector A: Magnitude=4, Angle=295 degrees
Vector B: Magnitude=7, Angle=12 degrees
Resultant Vector: Magnitude=8.80893, Angle=-14.2601 degrees

Pair 3:
Vector A: Magnitude=10, Angle=181 degrees
Vector B: Magnitude=3, Angle=67 degrees
Resultant Vector: Magnitude=9.1976, Angle=163.664 degrees

Pair 4:
Vector A: Magnitude=1, Angle=259 degrees
Vector B: Magnitude=4, Angle=166 degrees
Resultant Vector: Magnitude=4.07202, Angle=-179.804 degrees

Pair 5:
Vector A: Magnitude=1, Angle=306 degrees
Vector B: Magnitude=3, Angle=136 degrees
Resultant Vector: Magnitude=2.02266, Angle=140.925 degrees

Pair 6:
Vector A: Magnitude=2, Angle=8 degrees
Vector B: Magnitude=8, Angle=69 degrees
Resultant Vector: Magnitude=9.13859, Angle=57.9648 degrees

Pair 7:
Vector A: Magnitude=3,